# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv(r'G:\algorithmic-trading-python-master\\sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2031404952817,
 'week52high': 146.92,
 'week52low': 53.41,
 'week52highSplitAdjustOnly': 149.83,
 'week52lowSplitAdjustOnly': 54.43,
 'week52change': 1.1136423443018801,
 'sharesOutstanding': 17217464355,
 'float': 0,
 'avg10Volume': 126187100,
 'avg30Volume': 128589128,
 'day200MovingAvg': 125.15,
 'day50MovingAvg': 130.38,
 'employees': 152042,
 'ttmEPS': 3.79,
 'ttmDividendRate': 0.8201973114427342,
 'dividendYield': 0.007033693185138453,
 'nextDividendDate': '0',
 'exDividendDate': '2021-01-29',
 'nextEarningsDate': '2021-01-23',
 'peRatio': 32.09209672077756,
 'beta': 1.217634211458453,
 'maxChangePercent': 45.67609683482008,
 'year5ChangePercent': 4.005066406336574,
 'year2ChangePercent': 1.5325689630485475,
 'year1ChangePercent': 1.1457788935124444,
 'ytdChangePercent': -0.09587741885234714,
 'month6ChangePercent': 0.0938971511610177,
 'month3ChangePercent': -0.06476846414336569,
 'month1ChangePercent': -0.07713232091547895,
 'day30Chan

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

1.1457788935124444

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [8]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,124.89,0.894278,N/A
1,AAL,25.55,1.43517,N/A
2,AAP,186.00,1.5111,N/A
3,AAPL,120.08,1.12723,N/A
4,ABBV,108.48,0.590237,N/A
...,...,...,...,...
500,YUM,109.38,0.938593,N/A
501,ZBH,161.88,0.899835,N/A
502,ZBRA,478.88,1.58381,N/A
503,ZION,58.19,1.32386,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [9]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,VIAC,97.75,7.40891,N/A
1,FCX,36.47,5.60056,N/A
2,LB,60.63,5.38669,N/A
3,URI,320.00,3.65304,N/A
4,MGM,40.55,3.53754,N/A
5,GPS,32.45,3.42421,N/A
6,HAL,22.51,3.39928,N/A
7,FANG,78.47,3.24444,N/A
8,DISCA,79.75,3.1388,N/A
9,DFS,95.52,3.00017,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [10]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:1000000
1000000


In [11]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

C:\Users\Rajneesh\anaconda3\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,VIAC,97.75,7.40891,200
1,FCX,36.47,5.60056,537
2,LB,60.63,5.38669,323
3,URI,320.00,3.65304,61
4,MGM,40.55,3.53754,483
5,GPS,32.45,3.42421,604
6,HAL,22.51,3.39928,871
7,FANG,78.47,3.24444,249
8,DISCA,79.75,3.1388,245
9,DFS,95.52,3.00017,205


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [15]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [22]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row, f'{time_period} Price Return'] == None:
            hqm_dataframe.loc[row, f'{time_period} Price Return'] = 0.0
        #print(type(hqm_dataframe[f'{time_period} Price Return']))
        #print(type(hqm_dataframe.loc[row, f'{time_period} Price Return']))
        #print(stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100)
        
for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0      0.550495
1       0.80396
2      0.825743
3      0.706931
4      0.324752
         ...   
500    0.570297
501    0.564356
502    0.859406
503    0.760396
504    0.281188
Name: One-Year Return Percentile, Length: 505, dtype: object
0       0.493069
1       0.964356
2       0.508911
3       0.243564
4       0.394059
         ...    
500     0.425743
501      0.40396
502     0.912871
503     0.948515
504    0.0534653
Name: Six-Month Return Percentile, Length: 505, dtype: object
0       0.364356
1       0.980198
2       0.653465
3      0.0831683
4       0.243564
         ...    
500     0.213861
501     0.447525
502     0.782178
503      0.89505
504    0.0990099
Name: Three-Month Return Percentile, Length: 505, dtype: object
0       0.154455
1       0.992079
2       0.891089
3      0.0792079
4       0.221782
         ...    
500     0.435644
501     0.409901
502      0.10099
503     0.786139
504     0.136634
Name: One-Month Return Percentile, Length: 505, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,123.48,N/A,0.903975,0.550495,0.255415,0.493069,0.04443,0.364356,-0.034144,0.154455,N/A
1,AAL,26.02,N/A,1.46012,0.80396,1.06523,0.964356,0.556187,0.980198,0.338852,0.992079,N/A
2,AAP,190.00,N/A,1.53767,0.825743,0.268052,0.508911,0.16677,0.653465,0.127358,0.891089,N/A
3,AAPL,124.52,N/A,1.15207,0.706931,0.0970704,0.243564,-0.0644912,0.0831683,-0.0773173,0.0792079,N/A
4,ABBV,107.21,N/A,0.600438,0.324752,0.1941,0.394059,0.00888151,0.243564,-0.0156534,0.221782,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,108.59,N/A,0.917792,0.570297,0.225267,0.425743,-0.00144596,0.213861,0.0277328,0.435644,N/A
501,ZBH,162.54,N/A,0.913226,0.564356,0.200346,0.40396,0.0802319,0.447525,0.0231644,0.409901,N/A
502,ZBRA,460.06,N/A,1.60617,0.859406,0.866019,0.912871,0.231023,0.782178,-0.0658382,0.10099,N/A
503,ZION,56.87,N/A,1.31416,0.760396,0.961142,0.948515,0.34171,0.89505,0.0960249,0.786139,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [23]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [24]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [25]:
portfolio_input()

Enter the value of your portfolio:100000


In [26]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

C:\Users\Rajneesh\anaconda3\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,123.48,15,0.903975,0.550495,0.255415,0.493069,0.04443,0.364356,-0.034144,0.154455,0.390594
1,AAL,26.02,75,1.46012,0.80396,1.06523,0.964356,0.556187,0.980198,0.338852,0.992079,0.935149
2,AAP,190.00,10,1.53767,0.825743,0.268052,0.508911,0.16677,0.653465,0.127358,0.891089,0.719802
3,AAPL,124.52,15,1.15207,0.706931,0.0970704,0.243564,-0.0644912,0.0831683,-0.0773173,0.0792079,0.278218
4,ABBV,107.21,18,0.600438,0.324752,0.1941,0.394059,0.00888151,0.243564,-0.0156534,0.221782,0.29604
5,ABC,116.02,16,0.475196,0.239604,0.225034,0.423762,0.196028,0.710891,0.0960744,0.788119,0.540594
6,ABMD,311.52,6,1.35927,0.774257,0.13239,0.306931,0.0141886,0.261386,-0.0983681,0.0475248,0.347525
7,ABT,122.62,15,0.813715,0.487129,0.143564,0.326733,0.122648,0.558416,-0.0230501,0.194059,0.391584
8,ACN,276.14,7,0.807928,0.483168,0.140259,0.316832,0.012752,0.251485,0.0378964,0.491089,0.385644
9,ADBE,452.20,4,0.50219,0.271287,-0.0745357,0.0277228,-0.116907,0.0316832,-0.0813655,0.0732673,0.10099


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [27]:
writer = pd.ExcelWriter(r'G:\algorithmic-trading-python-master\momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [28]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [29]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [30]:
writer.save()